# Day18_0: Transformer 어텐션 메커니즘

## 📚 학습 목표

**Part 1: 기초**
1. Attention 메커니즘의 필요성 이해하기
2. Self-Attention 개념 이해하기
3. Query, Key, Value 이해하기
4. Scaled Dot-Product Attention 구현하기
5. 어텐션 가중치 시각화하기

**Part 2: 심화**
1. Multi-Head Attention 이해하기
2. Positional Encoding 이해하기
3. Transformer Encoder 구조 이해하기
4. PyTorch nn.Transformer 활용하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| Attention | 시퀀스 내 관계 파악 | 기계 번역, 요약, QA |
| Self-Attention | 문맥 이해 | "it"이 무엇을 가리키는지 파악 |
| Multi-Head | 다양한 관점 분석 | 문법적 관계, 의미적 관계 동시 파악 |
| Transformer | 현대 NLP의 기반 | GPT, BERT, LLM의 핵심 |

**분석가 관점**: Transformer는 현대 AI의 핵심입니다. ChatGPT, Claude, Gemini 모두 Transformer 기반이며, NLP부터 이미지, 비디오까지 거의 모든 분야에서 사용됩니다!

---

# Part 1: 기초

---

## 1.1 왜 Attention인가? - RNN의 한계

### RNN/LSTM의 한계

Day 17에서 배운 RNN, LSTM, GRU는 시퀀스 데이터를 잘 처리하지만, 치명적인 약점이 있습니다.

| 문제 | 설명 | 결과 |
|-----|------|------|
| 순차 처리 | t 시점이 끝나야 t+1 처리 가능 | GPU 병렬화 불가, 학습 느림 |
| 장기 의존성 | 정보가 한 단계씩 전달 | 긴 문장에서 앞 정보 손실 |
| 고정 Context | 마지막 hidden state만 사용 | 정보 병목 현상 |

### 예시: "it"은 무엇을 가리킬까?

```
"The animal didn't cross the street because it was too tired."
```

- "it"이 "animal"을 가리킨다는 것을 RNN이 어떻게 알까요?
- 두 단어가 멀리 떨어져 있으면, 연결 고리가 희미해집니다.

In [1]:
# RNN의 순차 처리 문제 시각화
import torch
import torch.nn as nn
import numpy as np

# 문장 예시
sentence = "The animal didn't cross the street because it was too tired".split()
print(f"문장: {sentence}")
print(f"단어 수: {len(sentence)}")
print(f"\n'it' 위치: {sentence.index('it')}")
print(f"'animal' 위치: {sentence.index('animal')}")
print(f"\n거리: {sentence.index('it') - sentence.index('animal')} 단어 떨어져 있음")

문장: ['The', 'animal', "didn't", 'cross', 'the', 'street', 'because', 'it', 'was', 'too', 'tired']
단어 수: 11

'it' 위치: 7
'animal' 위치: 1

거리: 6 단어 떨어져 있음


### Transformer의 해결책: Attention

**"Attention Is All You Need"** (2017, Google)

Transformer는 RNN의 순환 구조를 버리고, **Attention**만으로 모든 단어 관계를 한 번에 파악합니다.

| 특징 | RNN | Transformer |
|-----|-----|-------------|
| 처리 방식 | 순차적 | 병렬 |
| 장거리 의존성 | 어려움 | 직접 연결 |
| 학습 속도 | 느림 | 빠름 |
| 메모리 | O(n) | O(n^2) |

In [2]:
# RNN vs Transformer 비교
print("RNN 처리 방식:")
print("단어1 -> 단어2 -> 단어3 -> ... -> 단어n")
print("(순차적으로 하나씩 처리)\n")

print("Transformer 처리 방식:")
print("단어1 ←→ 단어2 ←→ 단어3 ←→ ... ←→ 단어n")
print("(모든 단어가 동시에 서로 참조)")

RNN 처리 방식:
단어1 -> 단어2 -> 단어3 -> ... -> 단어n
(순차적으로 하나씩 처리)

Transformer 처리 방식:
단어1 ←→ 단어2 ←→ 단어3 ←→ ... ←→ 단어n
(모든 단어가 동시에 서로 참조)


---

## 1.2 Self-Attention 개념

### Self-Attention이란?

**Self-Attention**: 같은 시퀀스 내에서 각 요소가 다른 모든 요소와의 관계를 계산하는 메커니즘

```
"The cat sat on the mat"

"cat"의 관점에서:
- "The"와의 관계: 0.1 (관사)
- "sat"과의 관계: 0.4 (주어-동사)
- "on"과의 관계: 0.1
- "the"와의 관계: 0.1
- "mat"과의 관계: 0.3 (의미적 연관)
```

### 비유: 회의에서 발표하기

| 역할 | 설명 |
|-----|------|
| 발표자 (Query) | "나는 이런 정보가 필요해" |
| 청중 (Key) | "나는 이런 정보를 가지고 있어" |
| 내용 (Value) | "그 정보의 실제 내용은 이거야" |

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Self-Attention 개념 시뮬레이션
words = ["The", "cat", "sat", "on", "the", "mat"]
print("문장:", " ".join(words))
print("\n각 단어는 다른 모든 단어와의 '관련도'를 계산합니다.")
print("\n예: 'cat'이 다른 단어들을 얼마나 '주목'해야 하는지")

문장: The cat sat on the mat

각 단어는 다른 모든 단어와의 '관련도'를 계산합니다.

예: 'cat'이 다른 단어들을 얼마나 '주목'해야 하는지


---

## 1.3 Query, Key, Value 이해하기

### Q, K, V란?

각 단어는 3가지 벡터로 변환됩니다:

| 벡터 | 역할 | 비유 |
|-----|------|------|
| **Query (Q)** | 내가 찾는 정보 | 검색어 |
| **Key (K)** | 내가 가진 정보의 제목 | 문서 제목 |
| **Value (V)** | 내가 가진 실제 정보 | 문서 내용 |

### 검색 엔진 비유

```
검색어 (Query): "고양이 사진"
   ↓
문서들의 Key와 비교
   ↓
관련도 높은 문서의 Value 가져오기
```

### 수식

$$Q = X \cdot W_Q$$
$$K = X \cdot W_K$$
$$V = X \cdot W_V$$

- $X$: 입력 임베딩 (시퀀스 길이 x 임베딩 차원)
- $W_Q, W_K, W_V$: 학습 가능한 가중치 행렬

In [4]:
# Q, K, V 생성 과정 시각화
embed_dim = 64
seq_len = 6
batch_size = 1

# 입력 임베딩 (단어들의 벡터 표현)
X = torch.randn(batch_size, seq_len, embed_dim)

# Q, K, V 변환을 위한 선형 레이어
W_Q = nn.Linear(embed_dim, embed_dim, bias=False)
W_K = nn.Linear(embed_dim, embed_dim, bias=False)
W_V = nn.Linear(embed_dim, embed_dim, bias=False)

# Q, K, V 생성
Q = W_Q(X)
K = W_K(X)
V = W_V(X)

print(f"입력 X shape: {X.shape}")
print(f"Query Q shape: {Q.shape}")
print(f"Key K shape: {K.shape}")
print(f"Value V shape: {V.shape}")

입력 X shape: torch.Size([1, 6, 64])
Query Q shape: torch.Size([1, 6, 64])
Key K shape: torch.Size([1, 6, 64])
Value V shape: torch.Size([1, 6, 64])


---

## 1.4 Scaled Dot-Product Attention 구현하기

### 어텐션 계산 4단계

```
1. Score 계산: Q @ K^T
2. 스케일링: Score / sqrt(d_k)
3. 가중치 계산: softmax(Score)
4. 가중합: weights @ V
```

### 수식

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### 왜 스케일링?

- Q와 K의 내적 값이 너무 커지면 softmax가 극단적인 값 (0 또는 1)을 출력
- $\sqrt{d_k}$로 나눠서 값의 분산을 안정화

In [5]:
def scaled_dot_product_attention(Q, K, V):
    """
    Scaled Dot-Product Attention 구현
    
    Args:
        Q: Query (batch, seq_len, d_k)
        K: Key (batch, seq_len, d_k)
        V: Value (batch, seq_len, d_v)
    
    Returns:
        output: 어텐션 적용 결과
        attention_weights: 어텐션 가중치
    """
    d_k = Q.size(-1)  # Key 차원
    
    # 1. Score 계산: Q @ K^T
    scores = torch.matmul(Q, K.transpose(-2, -1))
    print(f"1. Score (Q @ K^T) shape: {scores.shape}")
    
    # 2. 스케일링: Score / sqrt(d_k)
    scaled_scores = scores / math.sqrt(d_k)
    print(f"2. Scaled Score shape: {scaled_scores.shape}")
    
    # 3. 가중치 계산: softmax
    attention_weights = F.softmax(scaled_scores, dim=-1)
    print(f"3. Attention Weights shape: {attention_weights.shape}")
    
    # 4. 가중합: weights @ V
    output = torch.matmul(attention_weights, V)
    print(f"4. Output shape: {output.shape}")
    
    return output, attention_weights

# 테스트
print("=" * 50)
print("Scaled Dot-Product Attention 단계별 실행")
print("=" * 50)
output, weights = scaled_dot_product_attention(Q, K, V)

Scaled Dot-Product Attention 단계별 실행
1. Score (Q @ K^T) shape: torch.Size([1, 6, 6])
2. Scaled Score shape: torch.Size([1, 6, 6])
3. Attention Weights shape: torch.Size([1, 6, 6])
4. Output shape: torch.Size([1, 6, 64])


### Self-Attention 아키텍처

```mermaid
flowchart LR
    Input["Input<br/>(batch, seq_len, embed_dim)"] --> Q["Q Linear<br/>embed_dim → embed_dim"]
    Input --> K["K Linear<br/>embed_dim → embed_dim"]
    Input --> V["V Linear<br/>embed_dim → embed_dim"]
    Q --> Scores["Scores<br/>Q @ K^T / sqrt(d_k)"]
    K --> Scores
    Scores --> Softmax["Softmax"]
    Softmax --> Attn["Attention Weights"]
    V --> Output["Output<br/>Attn @ V"]
    Attn --> Output
    
    style Input fill:#ffffff,color:#000000
    style Q fill:#ffffff,color:#000000
    style K fill:#ffffff,color:#000000
    style V fill:#ffffff,color:#000000
    style Scores fill:#ffffff,color:#000000
    style Softmax fill:#ffffff,color:#000000
    style Attn fill:#ffffff,color:#000000
    style Output fill:#ffffff,color:#000000
```

In [6]:
# Self-Attention 클래스로 정리
class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        
        # Q, K, V 변환 레이어
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        # x: (batch, seq_len, embed_dim)
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        # Scaled Dot-Product Attention
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights

# 테스트
attention = SelfAttention(embed_dim=64)
x = torch.randn(1, 6, 64)  # (batch, seq_len, embed_dim)
output, weights = attention(x)

print(f"입력 shape: {x.shape}")
print(f"출력 shape: {output.shape}")
print(f"어텐션 가중치 shape: {weights.shape}")

입력 shape: torch.Size([1, 6, 64])
출력 shape: torch.Size([1, 6, 64])
어텐션 가중치 shape: torch.Size([1, 6, 6])


---

## 1.5 어텐션 가중치 시각화하기

### 💡 실무 예시: 어텐션 시각화로 모델 해석

어텐션 가중치를 시각화하면 모델이 어떤 단어에 주목하는지 알 수 있습니다.

In [7]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# 예시 문장
words = ["The", "cat", "sat", "on", "the", "mat"]

# Self-Attention 계산
attention = SelfAttention(embed_dim=64)
x = torch.randn(1, len(words), 64)
_, attention_weights = attention(x)

# 어텐션 가중치 추출
attn_matrix = attention_weights.squeeze().detach().numpy()

# Plotly 히트맵 시각화
fig = px.imshow(
    attn_matrix,
    labels=dict(x="Key (참조되는 단어)", y="Query (주목하는 단어)", color="가중치"),
    x=words,
    y=words,
    color_continuous_scale="Blues",
    aspect="equal"
)

fig.update_layout(
    title="Self-Attention 가중치 히트맵",
    xaxis_title="Key (참조되는 단어)",
    yaxis_title="Query (주목하는 단어)"
)

fig.show()

In [8]:
# 특정 단어의 어텐션 분포 막대 그래프
target_word = "cat"
target_idx = words.index(target_word)

# target_word가 다른 단어들에 주목하는 정도
attn_for_target = attn_matrix[target_idx]

fig = px.bar(
    x=words,
    y=attn_for_target,
    labels={"x": "단어", "y": "어텐션 가중치"},
    title=f"'{target_word}'이 다른 단어들에 주목하는 정도",
    color=attn_for_target,
    color_continuous_scale="Blues"
)

fig.update_layout(showlegend=False)
fig.show()

---

# Part 2: 심화

---

## 2.1 Multi-Head Attention 이해하기

### 왜 Multi-Head?

하나의 어텐션만으로는 문장의 다양한 관계를 모두 포착하기 어렵습니다.

| Head | 학습할 수 있는 관계 |
|------|-------------------|
| Head 1 | 주어-동사 관계 |
| Head 2 | 수식어-피수식어 |
| Head 3 | 대명사-선행사 |
| Head 4 | 의미적 유사성 |

### Multi-Head Attention 구조

```
1. 입력을 여러 Head로 분할
2. 각 Head가 독립적으로 Attention 수행
3. 모든 Head 결과를 Concat
4. 선형 변환으로 원래 차원 복원
```

### 수식

$$\text{MultiHead}(Q, K, V) = \text{Concat}(head_1, ..., head_h)W^O$$
$$\text{where } head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$

### Multi-Head Attention 아키텍처

```mermaid
flowchart LR
    Input["Input<br/>(batch, seq_len, embed_dim)"] --> QKV["Q/K/V Linear<br/>embed_dim → embed_dim"]
    QKV --> Split["Split into<br/>num_heads"]
    Split --> Head1["Head 1<br/>Attention"]
    Split --> Head2["Head 2<br/>Attention"]
    Split --> HeadN["Head N<br/>Attention"]
    Head1 --> Concat["Concat<br/>All Heads"]
    Head2 --> Concat
    HeadN --> Concat
    Concat --> FC["Linear<br/>embed_dim → embed_dim"]
    FC --> Output["Output<br/>(batch, seq_len, embed_dim)"]
    
    style Input fill:#ffffff,color:#000000
    style QKV fill:#ffffff,color:#000000
    style Split fill:#ffffff,color:#000000
    style Head1 fill:#ffffff,color:#000000
    style Head2 fill:#ffffff,color:#000000
    style HeadN fill:#ffffff,color:#000000
    style Concat fill:#ffffff,color:#000000
    style FC fill:#ffffff,color:#000000
    style Output fill:#ffffff,color:#000000
```

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim은 num_heads로 나누어 떨어져야 합니다."
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads  # 각 헤드의 차원
        
        # Q, K, V를 한 번에 생성 (효율적)
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        
        # 최종 출력 변환
        self.fc_out = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        
        # 1. Q, K, V 생성
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        # 2. 헤드 수에 맞게 분할: (B, N, D) -> (B, h, N, d_k)
        Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # 3. 각 헤드별 Scaled Dot-Product Attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attention_weights = F.softmax(scores, dim=-1)
        context = torch.matmul(attention_weights, V)  # (B, h, N, d_k)
        
        # 4. 헤드 합치기: (B, h, N, d_k) -> (B, N, D)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.embed_dim)
        
        # 5. 최종 선형 변환
        output = self.fc_out(context)
        
        return output, attention_weights

# 테스트
mha = MultiHeadAttention(embed_dim=256, num_heads=8)
x = torch.randn(2, 10, 256)  # (batch, seq_len, embed_dim)
output, weights = mha(x)

print(f"입력 shape: {x.shape}")
print(f"출력 shape: {output.shape}")
print(f"어텐션 가중치 shape: {weights.shape}")
print(f"(batch, num_heads, seq_len, seq_len)")

입력 shape: torch.Size([2, 10, 256])
출력 shape: torch.Size([2, 10, 256])
어텐션 가중치 shape: torch.Size([2, 8, 10, 10])
(batch, num_heads, seq_len, seq_len)


In [10]:
# Multi-Head Attention 가중치 시각화 (여러 헤드)
from plotly.subplots import make_subplots

words = ["The", "cat", "sat", "on", "the", "mat"]
mha = MultiHeadAttention(embed_dim=64, num_heads=4)
x = torch.randn(1, len(words), 64)
_, weights = mha(x)

# 4개 헤드의 어텐션 시각화
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[f"Head {i+1}" for i in range(4)]
)

for i in range(4):
    row = i // 2 + 1
    col = i % 2 + 1
    
    head_weights = weights[0, i].detach().numpy()
    
    fig.add_trace(
        go.Heatmap(
            z=head_weights,
            x=words,
            y=words,
            colorscale="Blues",
            showscale=(i == 1)
        ),
        row=row, col=col
    )

fig.update_layout(
    title="Multi-Head Attention: 각 헤드별 어텐션 가중치",
    height=600
)
fig.show()

---

## 2.2 Positional Encoding 이해하기

### 왜 위치 정보가 필요한가?

Transformer는 모든 단어를 동시에 처리하므로 **순서 정보가 없습니다**.

```
"고양이가 쥐를 쫓는다" vs "쥐가 고양이를 쫓는다"
→ Attention만으로는 구분 불가!
```

### Positional Encoding

각 위치에 고유한 벡터를 더해서 순서 정보를 주입합니다.

**원 논문의 sin/cos 방식:**

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

- 짝수 차원: sin 함수
- 홀수 차원: cos 함수
- 각 위치마다 고유한 패턴 생성

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Positional Encoding 행렬 생성
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # 짝수 인덱스: sin, 홀수 인덱스: cos
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)  # 학습되지 않는 파라미터로 등록
    
    def forward(self, x):
        # x: (batch, seq_len, d_model)
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

# 테스트
pe = PositionalEncoding(d_model=64)
x = torch.randn(2, 10, 64)
output = pe(x)

print(f"입력 shape: {x.shape}")
print(f"출력 shape: {output.shape}")
print(f"입력과 출력이 다른가? {not torch.allclose(x, output)}")

입력 shape: torch.Size([2, 10, 64])
출력 shape: torch.Size([2, 10, 64])
입력과 출력이 다른가? True


In [12]:
# Positional Encoding 시각화
d_model = 64
max_len = 100

pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

# 히트맵
fig = px.imshow(
    pe.numpy()[:50, :32],  # 처음 50개 위치, 32개 차원
    labels=dict(x="차원 (Dimension)", y="위치 (Position)", color="값"),
    color_continuous_scale="RdBu",
    aspect="auto"
)
fig.update_layout(title="Positional Encoding 패턴")
fig.show()

### Transformer Encoder Layer 아키텍처

```mermaid
flowchart LR
    Input["Input<br/>(batch, seq_len, embed_dim)"] --> MHA["Multi-Head<br/>Attention"]
    MHA --> Add1["Add<br/>+"]
    Input --> Add1
    Add1 --> Norm1["LayerNorm"]
    Norm1 --> FFN["Feed-Forward<br/>Linear → ReLU → Linear"]
    FFN --> Add2["Add<br/>+"]
    Norm1 --> Add2
    Add2 --> Norm2["LayerNorm"]
    Norm2 --> Output["Output<br/>(batch, seq_len, embed_dim)"]
    
    style Input fill:#ffffff,color:#000000
    style MHA fill:#ffffff,color:#000000
    style Add1 fill:#ffffff,color:#000000
    style Norm1 fill:#ffffff,color:#000000
    style FFN fill:#ffffff,color:#000000
    style Add2 fill:#ffffff,color:#000000
    style Norm2 fill:#ffffff,color:#000000
    style Output fill:#ffffff,color:#000000
```

### Transformer Encoder 아키텍처

```mermaid
flowchart LR
    Input["Input<br/>(batch, seq_len)<br/>token indices"] --> Embed["Embedding<br/>vocab_size → embed_dim"]
    Embed --> PosEnc["Positional<br/>Encoding"]
    PosEnc --> Layer1["Encoder Layer 1"]
    Layer1 --> Layer2["Encoder Layer 2"]
    Layer2 --> LayerN["Encoder Layer N"]
    LayerN --> Output["Output<br/>(batch, seq_len, embed_dim)"]
    
    style Input fill:#ffffff,color:#000000
    style Embed fill:#ffffff,color:#000000
    style PosEnc fill:#ffffff,color:#000000
    style Layer1 fill:#ffffff,color:#000000
    style Layer2 fill:#ffffff,color:#000000
    style LayerN fill:#ffffff,color:#000000
    style Output fill:#ffffff,color:#000000
```

---

## 2.3 Transformer Encoder 구조 이해하기

### Encoder Layer 구조

```
입력
 ↓
[Multi-Head Attention]
 ↓
[Add & Norm] ← 잔차 연결 + Layer Normalization
 ↓
[Feed-Forward Network]
 ↓
[Add & Norm]
 ↓
출력
```

### 핵심 구성 요소

| 구성요소 | 역할 |
|---------|------|
| Multi-Head Attention | 문맥 정보 추출 |
| Add & Norm | 학습 안정화 (잔차 연결 + 정규화) |
| Feed-Forward Network | 비선형 변환 (각 위치 독립적) |

In [13]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        
        # Multi-Head Attention
        self.attention = MultiHeadAttention(embed_dim, num_heads)
        
        # Layer Normalization
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        
        # Feed-Forward Network
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_dim, embed_dim)
        )
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # 1. Multi-Head Attention
        attn_output, _ = self.attention(x)
        
        # 2. Add & Norm (잔차 연결 + LayerNorm)
        x = self.norm1(x + self.dropout(attn_output))
        
        # 3. Feed-Forward Network
        ff_output = self.feed_forward(x)
        
        # 4. Add & Norm
        x = self.norm2(x + self.dropout(ff_output))
        
        return x

# 테스트
encoder_layer = TransformerEncoderLayer(
    embed_dim=256,
    num_heads=8,
    ff_dim=1024
)

x = torch.randn(2, 10, 256)
output = encoder_layer(x)

print(f"입력 shape: {x.shape}")
print(f"출력 shape: {output.shape}")

입력 shape: torch.Size([2, 10, 256])
출력 shape: torch.Size([2, 10, 256])


In [14]:
# 전체 Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim, num_layers, max_len=512, dropout=0.1):
        super().__init__()
        
        # 임베딩
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len, dropout)
        
        # Encoder 레이어들
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(embed_dim, num_heads, ff_dim, dropout)
            for _ in range(num_layers)
        ])
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # x: (batch, seq_len) - 토큰 인덱스
        
        # 임베딩 + Positional Encoding
        x = self.embedding(x)
        x = self.pos_encoding(x)
        
        # Encoder 레이어 통과
        for layer in self.layers:
            x = layer(x)
        
        return x

# 테스트
encoder = TransformerEncoder(
    vocab_size=10000,
    embed_dim=256,
    num_heads=8,
    ff_dim=1024,
    num_layers=6
)

x = torch.randint(0, 10000, (2, 50))  # (batch, seq_len)
output = encoder(x)

print(f"입력 shape: {x.shape} (토큰 인덱스)")
print(f"출력 shape: {output.shape} (문맥 벡터)")
print(f"\n총 파라미터: {sum(p.numel() for p in encoder.parameters()):,}")

입력 shape: torch.Size([2, 50]) (토큰 인덱스)
출력 shape: torch.Size([2, 50, 256]) (문맥 벡터)

총 파라미터: 7,298,560


---

## 2.4 PyTorch nn.Transformer 활용하기

### PyTorch 내장 Transformer

PyTorch는 Transformer 관련 모듈을 제공합니다:

| 모듈 | 설명 |
|-----|------|
| `nn.MultiheadAttention` | Multi-Head Attention |
| `nn.TransformerEncoderLayer` | Encoder 레이어 1개 |
| `nn.TransformerEncoder` | Encoder 스택 |
| `nn.Transformer` | 전체 Encoder-Decoder |

### Transformer 분류 모델 아키텍처

```mermaid
flowchart LR
    Input["Input<br/>(batch, seq_len)<br/>token indices"] --> Embed["Embedding<br/>vocab_size → embed_dim"]
    Embed --> PosEnc["Positional<br/>Encoding"]
    PosEnc --> Encoder["Transformer<br/>Encoder<br/>(N layers)"]
    Encoder --> EncOut["(batch, seq_len, embed_dim)"]
    EncOut --> MeanPool["Mean Pooling<br/>mean(dim=1)"]
    MeanPool --> PoolOut["(batch, embed_dim)"]
    PoolOut --> FC1["Linear<br/>embed_dim → embed_dim//2"]
    FC1 --> ReLU["ReLU"]
    ReLU --> Dropout["Dropout"]
    Dropout --> FC2["Linear<br/>embed_dim//2 → num_classes"]
    FC2 --> Output["Output<br/>(batch, num_classes)"]
    
    style Input fill:#ffffff,color:#000000
    style Embed fill:#ffffff,color:#000000
    style PosEnc fill:#ffffff,color:#000000
    style Encoder fill:#ffffff,color:#000000
    style EncOut fill:#ffffff,color:#000000
    style MeanPool fill:#ffffff,color:#000000
    style PoolOut fill:#ffffff,color:#000000
    style FC1 fill:#ffffff,color:#000000
    style ReLU fill:#ffffff,color:#000000
    style Dropout fill:#ffffff,color:#000000
    style FC2 fill:#ffffff,color:#000000
    style Output fill:#ffffff,color:#000000
```

In [15]:
# PyTorch 내장 nn.MultiheadAttention
mha = nn.MultiheadAttention(embed_dim=256, num_heads=8, batch_first=True)

x = torch.randn(2, 10, 256)  # (batch, seq_len, embed_dim)
output, weights = mha(x, x, x)  # Self-Attention: Q=K=V=x

print(f"nn.MultiheadAttention:")
print(f"  입력: {x.shape}")
print(f"  출력: {output.shape}")
print(f"  가중치: {weights.shape}")

nn.MultiheadAttention:
  입력: torch.Size([2, 10, 256])
  출력: torch.Size([2, 10, 256])
  가중치: torch.Size([2, 10, 10])


In [16]:
# PyTorch 내장 nn.TransformerEncoderLayer
encoder_layer = nn.TransformerEncoderLayer(
    d_model=256,
    nhead=8,
    dim_feedforward=1024,
    dropout=0.1,
    batch_first=True
)

x = torch.randn(2, 10, 256)
output = encoder_layer(x)

print(f"nn.TransformerEncoderLayer:")
print(f"  입력: {x.shape}")
print(f"  출력: {output.shape}")

nn.TransformerEncoderLayer:
  입력: torch.Size([2, 10, 256])
  출력: torch.Size([2, 10, 256])


In [17]:
# PyTorch 내장 nn.TransformerEncoder
encoder_layer = nn.TransformerEncoderLayer(
    d_model=256, nhead=8, dim_feedforward=1024, batch_first=True
)
encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

x = torch.randn(2, 10, 256)
output = encoder(x)

print(f"nn.TransformerEncoder (6 layers):")
print(f"  입력: {x.shape}")
print(f"  출력: {output.shape}")

nn.TransformerEncoder (6 layers):
  입력: torch.Size([2, 10, 256])
  출력: torch.Size([2, 10, 256])


In [18]:
# 텍스트 분류를 위한 Transformer 모델
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim, num_layers, num_classes, max_len=512, dropout=0.1):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len, dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim // 2, num_classes)
        )
    
    def forward(self, x):
        # x: (batch, seq_len)
        x = self.embedding(x)
        x = self.pos_encoding(x)
        x = self.encoder(x)
        
        # 평균 풀링 (또는 [CLS] 토큰 사용 가능)
        x = x.mean(dim=1)  # (batch, embed_dim)
        x = self.classifier(x)
        
        return x

# 테스트
model = TransformerClassifier(
    vocab_size=10000,
    embed_dim=128,
    num_heads=4,
    ff_dim=512,
    num_layers=2,
    num_classes=2
)

x = torch.randint(0, 10000, (4, 100))  # (batch, seq_len)
output = model(x)

print(f"Transformer 텍스트 분류 모델:")
print(f"  입력: {x.shape}")
print(f"  출력: {output.shape}")
print(f"\n총 파라미터: {sum(p.numel() for p in model.parameters()):,}")

Transformer 텍스트 분류 모델:
  입력: torch.Size([4, 100])
  출력: torch.Size([4, 2])

총 파라미터: 1,684,930


### 💡 실무 예시: IMDB 감성 분석 (간단 버전)

In [19]:
# IMDB 데이터셋 로드 (torchtext 사용)
# 참고: 실제 학습은 시간이 오래 걸리므로 구조만 확인합니다

from torch.utils.data import DataLoader, TensorDataset

# 가상의 데이터 생성 (실제로는 IMDB 데이터 사용)
vocab_size = 10000
seq_len = 200
num_samples = 1000

# 랜덤 데이터 (실제로는 토큰화된 리뷰 텍스트)
X_train = torch.randint(0, vocab_size, (num_samples, seq_len))
y_train = torch.randint(0, 2, (num_samples,))  # 0: 부정, 1: 긍정

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"학습 데이터: {len(train_dataset)} 샘플")
print(f"배치 수: {len(train_loader)}")

학습 데이터: 1000 샘플
배치 수: 32


In [20]:
# 모델, 손실 함수, 옵티마이저 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

model = TransformerClassifier(
    vocab_size=vocab_size,
    embed_dim=128,
    num_heads=4,
    ff_dim=256,
    num_layers=2,
    num_classes=2
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"모델 파라미터: {sum(p.numel() for p in model.parameters()):,}")

Device: cpu
모델 파라미터: 1,553,346


In [21]:
# 간단한 학습 루프 (1 에포크)
model.train()
total_loss = 0
correct = 0
total = 0

for batch_idx, (texts, labels) in enumerate(train_loader):
    texts, labels = texts.to(device), labels.to(device)
    
    optimizer.zero_grad()
    outputs = model(texts)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    total_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    
    if batch_idx % 10 == 0:
        print(f"Batch {batch_idx}/{len(train_loader)}: Loss={loss.item():.4f}")

print(f"\n에포크 완료: Loss={total_loss/len(train_loader):.4f}, Acc={100.*correct/total:.2f}%")

Batch 0/32: Loss=0.7034
Batch 10/32: Loss=0.7418
Batch 20/32: Loss=0.6772
Batch 30/32: Loss=0.7162

에포크 완료: Loss=0.7030, Acc=51.00%


---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. Attention Score 계산 ⭐

**문제**: Query와 Key가 주어졌을 때, Attention Score를 계산하세요.

```python
Q = torch.tensor([[1.0, 0.0], [0.0, 1.0]])  # (2, 2)
K = torch.tensor([[1.0, 1.0], [0.0, 1.0]])  # (2, 2)
```

힌트: Score = Q @ K^T

In [22]:
# 여기에 코드를 작성하세요


### Q2. Scaled Dot-Product Attention ⭐

**문제**: Score를 스케일링하고 softmax를 적용하여 어텐션 가중치를 계산하세요.

d_k = 2 (Key 차원)

In [23]:
# 여기에 코드를 작성하세요


### Q3. Query, Key, Value 생성 ⭐⭐

**문제**: 입력 텐서 X로부터 Q, K, V를 생성하는 선형 레이어를 정의하고 적용하세요.

- 입력: X (batch=1, seq_len=4, embed_dim=8)
- 출력: Q, K, V 각각 (1, 4, 8)

In [24]:
# 여기에 코드를 작성하세요


### Q4. Self-Attention 전체 계산 ⭐⭐

**문제**: Q, K, V가 주어졌을 때, Self-Attention의 전체 계산을 수행하세요.

1. Score 계산
2. 스케일링
3. Softmax
4. Value와 가중합

In [25]:
# 여기에 코드를 작성하세요


### Q5. 어텐션 가중치 시각화 ⭐⭐

**문제**: 5개 단어로 구성된 문장의 Self-Attention 가중치를 히트맵으로 시각화하세요.

단어: ["I", "love", "deep", "learning", "!"]

In [26]:
# 여기에 코드를 작성하세요


### Q6. Multi-Head Attention 헤드 분할 ⭐⭐⭐

**문제**: embed_dim=64, num_heads=4일 때, 텐서를 헤드별로 분할하세요.

- 입력: (batch=2, seq_len=10, embed_dim=64)
- 출력: (batch=2, num_heads=4, seq_len=10, head_dim=16)

In [27]:
# 여기에 코드를 작성하세요


### Q7. Positional Encoding 생성 ⭐⭐⭐

**문제**: d_model=16, max_len=10인 Positional Encoding을 생성하고 시각화하세요.

In [28]:
# 여기에 코드를 작성하세요


### Q8. Transformer Encoder Layer 구현 ⭐⭐⭐⭐

**문제**: Multi-Head Attention, Add & Norm, Feed-Forward를 포함하는 Encoder Layer를 구현하세요.

설정:
- embed_dim = 64
- num_heads = 4
- ff_dim = 256

In [29]:
# 여기에 코드를 작성하세요


### Q9. nn.TransformerEncoder 활용 ⭐⭐⭐⭐

**문제**: PyTorch의 nn.TransformerEncoder를 사용하여 6개 레이어의 Encoder를 만들고 순전파를 수행하세요.

In [30]:
# 여기에 코드를 작성하세요


### Q10. Transformer 텍스트 분류 모델 ⭐⭐⭐⭐⭐

**문제**: Transformer 기반 텍스트 분류 모델을 구현하고, 가상의 데이터로 1 에포크 학습을 수행하세요.

요구사항:
1. Embedding + Positional Encoding
2. TransformerEncoder (2 layers)
3. 평균 풀링 + 분류 헤드
4. 학습 루프 (1 에포크)

In [31]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 수식/코드 | 역할 |
|-----|---------------|------|
| Self-Attention | Q @ K^T @ V | 시퀀스 내 관계 파악 |
| Q, K, V | X @ W_Q, X @ W_K, X @ W_V | 질의, 키, 값 변환 |
| Scaling | / sqrt(d_k) | 값 안정화 |
| Softmax | F.softmax(scores, dim=-1) | 가중치 정규화 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 | 효과 |
|-----|------|------|
| Multi-Head | 여러 관점 병렬 처리 | 다양한 관계 포착 |
| Positional Encoding | sin/cos 패턴 | 순서 정보 주입 |
| Add & Norm | 잔차 연결 + LayerNorm | 학습 안정화 |
| Feed-Forward | 2층 MLP | 비선형 변환 |

### Attention 공식

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### 💡 실무 팁

1. **Multi-Head 수**: embed_dim을 나누어 떨어지게 설정 (보통 8, 16)
2. **ff_dim**: 일반적으로 embed_dim의 4배 (예: 512 -> 2048)
3. **Layer 수**: 작은 모델 2-4, 큰 모델 6-12+
4. **Dropout**: 0.1이 기본, 과적합 시 0.2-0.3
5. **메모리**: Attention은 O(n^2), 긴 시퀀스 주의